In [1]:
##IMPORT DEPENDENCIES

import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import re
import random
from datetime import datetime
import matplotlib.pyplot as plt
from flask import jsonify
import sqlite3
import time


*****************************
SCRAPE FLIXPATROL
*****************************

In [44]:
#SET UP SPLINTER BROWSER ENGINGE
executable_path= {'executable_path': ChromeDriverManager().install()}
browser= Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST driver version for 100.0.4896
Driver [/Users/juliankauffmann/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache


In [38]:
#FUNCTION TO OPEN MAIN FLIX SITE
def open_flix(the_url):
    browser.visit(the_url)

In [39]:
#PARSES HTML
def get_updated_html():
    thesoup = BeautifulSoup(browser.html, 'html.parser')

    return thesoup

In [46]:
#FUNCTION - BOT BROWSES SITE AND PULLS DATA

def web_bot(generation,gender,country):
    titles=[]
    the_html=get_updated_html()
    temp_titles=the_html.find_all("div", class_="group-hover:underline")

    for title in temp_titles:
        titles.append(title.text.split('\n')[1].split('\t')[9])

    #get views of top 100
    views=[]

    temp_views=the_html.find_all("td", class_="table-td text-right text-gray-400 font-semibold tabular-nums")

    for view in temp_views:
        views.append(view.text)

    #get category of top 100
    categories=[]

    temp_cats=the_html.find_all("div", class_="flex flex-wrap text-sm leading-6 text-gray-500")

    for cat in temp_cats:
        for c in cat.span:
            categories.append(c)

    #get country, premiere date, and genre info of top 100
    countries=[]
    premieres=[]
    genres=[]

    temp_infos=the_html.find_all("div", class_="flex flex-wrap text-sm leading-6 text-gray-500")

    #adds a delineator to genres which will be parsed in JavaScript
    for info in temp_infos:
        temp_genre=""
        for i in range(0,len(info.text.split('\n'))-1):
            temp=info.text.split('\n')[i]       
            if i>6:
                if (temp !="|") and (temp !=""):
                    temp_genre=temp_genre+'|'+temp            

        genres.append(temp_genre)
        premieres.append(info.text.split('\n')[5])
        countries.append(info.text.split('\n')[3])

    return pd.DataFrame({"title":titles,"view":views,"category":categories,
                         "country":countries,"premiere":premieres, "genre":genres,
                         "generation":generation,"gender":gender,"viewing_country":country})
    

In [43]:
#FUNCTION - HAS BOT ITERATE THROUGH THE DROP-DOWN OPTIONS
def iterate_through_site():
    
    #SET VARIABLES
    #creates empty dataframe
    demographics_df=pd.DataFrame({"title":[],"view":[],"category":[],
                  "country":[],"premiere":[], "genre":[],"generation":[],"gender":[],"viewing_country":[]})

    #DROP-DOWN LIST for bot to iterate through
    generations=['generation-z','baby-boomers','generation-y','generation-x']
    genders=['men','women']
    countries=['brazil','canada','france','germany','india','mexico','united-kingdom','united-states']

    #SCRAPE SITE
    for gender in genders:
        for country in countries:
            for generation in generations:
                the_url=f'https://flixpatrol.com/demographics/{generation}/{gender}/{country}/'
                open_flix(the_url)
                time.sleep(3)
                temp_df=web_bot(generation,gender,country)
                demographics_df=pd.concat([demographics_df,temp_df])
                
    return demographics_df


In [47]:
#INITIATE BOT
url='https://flixpatrol.com/demographics/'

open_flix(url)

time.sleep(4)

flix_df=iterate_through_site()

browser.quit()

In [48]:
#VERIFY RESULTING DATA
flix_df.head()

,title,view,category,country,premiere,genre,generation,gender,viewing_country
0,Breaking Bad,"300,000",TV Show,United States,01/20/2008,|Crime|Drug Cartel,generation-z,men,united-kingdom
1,Game of Thrones,"280,000",TV Show,United States,04/17/2011,| HBO|Fantasy|Dragons,generation-z,men,united-kingdom
2,Family Guy,"240,000",TV Show,United States,01/31/1999,|Animation|Adult Animation,generation-z,men,united-kingdom
3,The Simpsons,"230,000",TV Show,United States,12/17/1989,|Animation|Adult Animation,generation-z,men,united-kingdom
4,Vikings,"210,000",TV Show,Ireland,03/03/2013,|History|Vikings,generation-z,men,united-kingdom


*****************************
IMDB DATA
*****************************

In [2]:
#READ IN DATA
titles_url="title.basics.tsv"
ratings_url="title.ratings.tsv"

titles_df = pd.read_csv(titles_url, sep='\t')
ratings_df = pd.read_csv(ratings_url, sep='\t')

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
#VERIFY TITLES DF
titles_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [29]:
#VERIFY RATINGS DF
ratings_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1872
1,tt0000002,5.9,247
2,tt0000003,6.5,1646
3,tt0000004,5.8,160
4,tt0000005,6.2,2473


*****************************
EXPLORE AND CLEAN DATA
*****************************

In [3]:
#Find Length
titles_df.shape

(8846979, 9)

In [4]:
#Analyze isAdult column
print(titles_df['isAdult'].unique())

#Remove All Adult Title that do not equal 0
titles_df=titles_df.loc[titles_df['isAdult']==0]

[0 1 2019 1981 2020 2017 '0' '1' '\\N' 2014 2005]


In [5]:
#Analyze title type
print(titles_df['titleType'].unique())

#Just keep movie and tvSeries
titles_df=titles_df.loc[(titles_df['titleType']=='movie') | (titles_df['titleType']=='tvSeries')]

['short' 'movie' 'tvEpisode' 'tvSeries' 'tvShort' 'tvMovie' 'tvMiniSeries'
 'tvSpecial' 'video' 'videoGame' 'tvPilot']


In [6]:
#Check new length
titles_df.shape

(815140, 9)

In [11]:
#Delete Unneeded Columns
titles_df=titles_df[['tconst','titleType','primaryTitle','genres']]

In [13]:
#Rename Columns
titles_df.rename(columns={'titleType':'category','primaryTitle':'title'},inplace=True)
titles_df.head()

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,tconst,category,title,genres
498,tt0000502,movie,Bohemios,\N
570,tt0000574,movie,The Story of the Kelly Gang,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,Drama
610,tt0000615,movie,Robbery Under Arms,Drama
625,tt0000630,movie,Hamlet,Drama


In [16]:
#Read Ratings Table
print (f'table length: {ratings_df.shape}')
ratings_df.head()

table length: (1234159, 3)


,tconst,averageRating,numVotes
0,tt0000001,5.7,1872
1,tt0000002,5.9,247
2,tt0000003,6.5,1646
3,tt0000004,5.8,160
4,tt0000005,6.2,2473


In [18]:
#Merge the two tables based on common id
titles_merge=pd.merge(titles_df,ratings_df,on='tconst' , how='inner')
titles_merge.head()

,tconst,category,title,genres,averageRating,numVotes
0,tt0000502,movie,Bohemios,\N,4.5,14
1,tt0000574,movie,The Story of the Kelly Gang,"Action,Adventure,Biography",6.0,759
2,tt0000591,movie,The Prodigal Son,Drama,4.6,17
3,tt0000615,movie,Robbery Under Arms,Drama,4.5,23
4,tt0000630,movie,Hamlet,Drama,3.8,24


In [27]:
#Delete Nulls as described by IMDB - \N
titles_merge=titles_merge.loc[titles_merge['genres']!='\\N']
titles_merge.head()

,tconst,category,title,genres,averageRating,numVotes
1,tt0000574,movie,The Story of the Kelly Gang,"Action,Adventure,Biography",6.0,759
2,tt0000591,movie,The Prodigal Son,Drama,4.6,17
3,tt0000615,movie,Robbery Under Arms,Drama,4.5,23
4,tt0000630,movie,Hamlet,Drama,3.8,24
5,tt0000675,movie,Don Quijote,Drama,4.9,19


*****************************
SET UP SQLITE DB
*****************************

In [29]:
#SET UP SQL ENGINGE
connection = sqlite3.connect('IMDB.db')
conn=connection.cursor()

In [62]:
#CREATE TABLES FROM PANDAS DF
titles_merge.to_sql('titles',connection)
flix_df.to_sql('flix',connection)

In [56]:
connection.commit()

In [61]:
#VERIFY DATA
pd.read_sql('select * from titles',connection)

,index,tconst,category,title,genres,averageRating,numVotes
0,1,tt0000574,movie,The Story of the Kelly Gang,"Action,Adventure,Biography",6.0,759
1,2,tt0000591,movie,The Prodigal Son,Drama,4.6,17
2,3,tt0000615,movie,Robbery Under Arms,Drama,4.5,23
3,4,tt0000630,movie,Hamlet,Drama,3.8,24
4,5,tt0000675,movie,Don Quijote,Drama,4.9,19
...,...,...,...,...,...,...,...
338872,352139,tt9916270,movie,Il talento del calabrone,Thriller,5.8,1329
338873,352140,tt9916362,movie,Coven,"Drama,History",6.4,4542
338874,352141,tt9916380,tvSeries,Meie aasta Aafrikas,"Adventure,Comedy,Family",9.2,106
338875,352142,tt9916428,movie,The Secret of China,"Adventure,History,War",3.6,15


In [64]:
#Only keep rows where votes are greater than 1000
print(pd.read_sql('select count(numVotes) from titles where cast(numVotes as int)>1000',connection))

connection.execute('delete from titles where cast(numVotes as int)<1000')
connection.commit()

   count(numVotes)
0            43850


In [72]:
#Examine all data in one table
pd.read_sql('select f.*, t.numVotes,t.averageRating from flix f \
            join titles t on f.title=t.title',connection).head(5)

,index,title,view,category,country,premiere,genre,generation,gender,viewing_country,numVotes,averageRating
0,0,Breaking Bad,"300,000",TV Show,United States,01/20/2008,|Crime|Drug Cartel,generation-z,men,united-kingdom,1706281,9.5
1,1,Game of Thrones,"280,000",TV Show,United States,04/17/2011,| HBO|Fantasy|Dragons,generation-z,men,united-kingdom,1972994,9.3
2,2,Family Guy,"240,000",TV Show,United States,01/31/1999,|Animation|Adult Animation,generation-z,men,united-kingdom,325380,8.2
3,3,The Simpsons,"230,000",TV Show,United States,12/17/1989,|Animation|Adult Animation,generation-z,men,united-kingdom,393116,8.7
4,4,Vikings,"210,000",TV Show,Ireland,03/03/2013,|History|Vikings,generation-z,men,united-kingdom,504178,8.6


*****************************
MOVE TO GRAPH EXPLORATION
*****************************